In [1]:
import numpy as np

In [2]:
arr = {"Customer Order": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24] , 
         "RIAT": [0,94,73,70,82,25,35,61,42,48,26,88,31,90,55,95,58,70,15,73,65,74,75,98] , 
         "IAT": [] , "Clock": [] , 
         "RST": [32,96,89,32,67,48,63,99,98,66,85,58,6,39,15,2,48,63,85,61,40,16,18,52] ,
         "Able ST Begins": [] , "Able ST": [], "Able ST Ends": [], "Baker ST Begins": [],"Baker ST": [],
         "Baker ST Ends": [], "Queuing": [0],
         "Time Spend in System": [], "Able Idle": [], "Baker Idle": []}

IA = {1: [A for A in range(0,25)], 2: [A for A in range(25,65)], 3: [A for A in range(65,85)],
                 4: [A for A in range(85,100)]}

BakerST = {2: [A for A in range(0,30)], 3: [A for A in range(30,58)], 4: [A for A in range(58,83)],
                 5: [A for A in range(83,100)]}

AbleST = {2: [A for A in range(0,35)], 3: [A for A in range(35,60)], 4: [A for A in range(60,80)],
                 5: [A for A in range(80,100)]}

In [3]:
def IATClock(arr, IA, i):
    if i == 0:
        arr["Clock"].append(0)
        arr["IAT"].append(0)
    else:
        if arr["RIAT"][i] in IA[1]:
            arr["IAT"].append(1)
        elif arr["RIAT"][i] in IA[2]:
            arr["IAT"].append(2)
        elif arr["RIAT"][i] in IA[3]:
            arr["IAT"].append(3)
        else:
            arr["IAT"].append(4)
        CurrentClock = arr["Clock"][-1] + arr["IAT"][i]
        arr["Clock"].append(CurrentClock)
    return arr

In [4]:
def BakerOrAble(arr):
    if len(arr["Able ST Ends"]) == 0:
        return 1 
    elif len(arr["Baker ST Ends"]) == 0:
        if (arr["Clock"][-1] - arr["Able ST Ends"][-1]) < 0:
            return 0 
        else:
            return 1 
    elif arr["Able ST Ends"][-1] <= arr["Clock"][-1] or (arr["Able ST Ends"][-1] <= arr["Baker ST Ends"][-1]):
        return 1
    else:
        return 0

In [5]:
TotalTime = 0
i = 0
while (TotalTime <= 60):
    arr = IATClock(arr, IA, i)
    Result = BakerOrAble(arr)
    if Result == 1:
        if len(arr["Baker ST Ends"]) == 0:
            arr["Baker ST Begins"].append(arr["Clock"][-1])
        elif arr["Clock"][-1] > arr["Baker ST Ends"][-1]:
            arr["Baker ST Begins"].append(arr["Clock"][-1])
        else:
            arr["Baker ST Begins"].append(arr["Baker ST Ends"][-1])
        if arr["RST"][i] in BakerST[2]:
            arr["Baker ST"].append(2)
        elif  arr["RST"][i] in BakerST[3]:
            arr["Baker ST"].append(3)
        elif arr["RST"][i] in BakerST[4]:
            arr["Baker ST"].append(4)
        elif arr["RST"][i] in BakerST[5]:
            arr["Baker ST"].append(5)
        arr["Baker ST Ends"].append(arr["Baker ST Begins"][-1] + arr["Baker ST"][-1])
        arr["Queuing"].append(arr["Baker ST Begins"][-1] - arr["Clock"][-1])
        arr["Time Spend in System"].append(arr["Baker ST"][-1] + arr["Queuing"][-1])
        if len(arr["Baker ST Ends"]) == 1:
            arr["Baker Idle"].append(arr["Clock"][-1])
        else:
            arr["Baker Idle"].append(max(arr["Clock"][-1] - arr["Baker ST Ends"][-2],0))
    else:
        if len(arr["Able ST Ends"]) == 0:
            arr["Able ST Begins"].append(arr["Clock"][-1])
        elif arr["Clock"][-1] > arr["Able ST Ends"][-1]:
            arr["Able ST Begins"].append(arr["Clock"][-1])
        else:
            arr["Able ST Begins"].append(arr["Able ST Ends"][-1])
        if arr["RST"][i] in AbleST[2]:
            arr["Able ST"].append(2)
        elif arr["RST"][i] in AbleST[3]:
            arr["Able ST"].append(3)
        elif arr["RST"][i] in AbleST[4]:
            arr["Able ST"].append(4)
        else:
            arr["Able ST"].append(5)
        arr["Able ST Ends"].append(arr["Able ST Begins"][-1] + arr["Able ST"][-1])
        arr["Queuing"].append(arr["Able ST Begins"][-1] - arr["Clock"][-1])
        arr["Time Spend in System"].append(arr["Able ST"][-1] + arr["Queuing"][-1])
        if len(arr["Able ST Ends"]) == 1:
            arr["Able Idle"].append(arr["Clock"][-1])
        else:
            arr["Able Idle"].append(max(arr["Clock"][-1] - arr["Able ST Ends"][-2] , 0))
    i += 1
    TotalTime = arr["Clock"][-1]

In [6]:
print("Baker Service Time Begins : {}".format(arr["Baker ST Begins"]))
print("Baker Service Time : {}".format(arr["Baker ST"]))
print("Baker Service Time Ends : {}".format(arr["Baker ST Ends"]))

Baker Service Time Begins : [0, 4, 10, 13, 17, 21, 26, 31, 35, 41, 43, 46, 50, 56, 59, 63]
Baker Service Time : [2, 5, 2, 4, 4, 5, 5, 2, 3, 2, 3, 4, 4, 2, 2, 3]
Baker Service Time Ends : [2, 9, 12, 17, 21, 26, 31, 33, 38, 43, 46, 50, 54, 58, 61, 66]


In [7]:
print("Able Service Time Begins : {}".format(arr["Able ST Begins"]))
print("Able Service Time : {}".format(arr["Able ST"]))
print("Able Service Time Ends : {}".format(arr["Able ST Ends"]))

Able Service Time Begins : [7, 15, 19, 24, 29, 37, 47, 53]
Able Service Time : [5, 3, 5, 4, 4, 2, 5, 3]
Able Service Time Ends : [12, 18, 24, 28, 33, 39, 52, 56]


In [8]:
AbleBusyPr = round((sum(arr["Able ST"])) / (arr["Clock"][-2]) * 100,2)
print("Percentage for Able's Busy is {}%".format(AbleBusyPr))

Percentage for Able's Busy is 52.54%


In [9]:
WaitingTime = sum(arr["Queuing"]) / np.count_nonzero(arr["Queuing"])
print("Average Waiting Time is {} minutes".format(WaitingTime))

Average Waiting Time is 1.0 minutes
